In [1]:
import os

if not os.environ.get("KERAS_BACKEND"):
    # Set to your favorite backend
    os.environ["KERAS_BACKEND"] = "tensorflow"

In [2]:
import bayesflow as bf
import keras

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

INFO:bayesflow:Using backend 'tensorflow'


In [3]:
def prior():
    mu = np.random.standard_normal()
    return {"mu": mu}

In [4]:
def likelihood(mu, num_obs=10):
    return {"x": mu + np.random.standard_normal((num_obs, 1))}

In [5]:
simulator = bf.make_simulator([prior, likelihood])

In [6]:
adapter = bf.approximators.RatioApproximator.build_adapter(
    inference_variables=["mu"], 
    inference_conditions=["x"]
)

In [7]:
ratio_approximator = bf.approximators.RatioApproximator(
    adapter=adapter,
    classifier_network=bf.networks.MLP(),
    summary_network=bf.networks.DeepSet(),
    gamma=5
)

In [8]:
ratio_approximator.compile(optimizer="adam")

In [ ]:
history = ratio_approximator.fit(
    epochs=10, 
    simulator=simulator,
    num_batches=1000, 
    batch_size=32
)

INFO:bayesflow:Building dataset from simulator instance of SequentialSimulator.
INFO:bayesflow:Using 10 data loading workers.
INFO:bayesflow:Building on a test batch.


Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 27s 22ms/step - loss: 0.4806
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - loss: 0.4494
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 26s 26ms/step - loss: 0.4478
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 29s 29ms/step - loss: 0.4472
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 28s 28ms/step - loss: 0.4466
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 28s 28ms/step - loss: 0.4464
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 30s 30ms/step - loss: 0.4460
Epoch 8/10
 770/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 0.4458

In [ ]:
f = bf.diagnostics.plots.loss(history)

In [ ]:
sims = simulator.sample(100)
contrastive_sims = {
    "mu": sims["mu"][::-1],
    "x": sims["x"]
}

In [ ]:
log_ratio_positive = keras.ops.convert_to_numpy(ratio_approximator.log_ratio(sims))
log_ratio_negative = keras.ops.convert_to_numpy(ratio_approximator.log_ratio(contrastive_sims))

In [ ]:
f, ax = plt.subplots(1, 1)
sns.histplot(log_ratio_positive, ax=ax, legend="Positive", color="#00AA00", alpha=0.5)
sns.histplot(log_ratio_negative, ax=ax, legend="Positive", color="#AA0000", alpha=0.5)

## Verify likelihood-evidence ratio against likelihood 

The log diff of likelihood and likelihood-evidence ratio should be constant.

In [ ]:
def log_likelihood(x, mu):
    return np.sum(norm.logpdf(x, loc=mu, scale=1), axis=1)

In [ ]:
mu = sims["mu"]
x_obs = sims["x"][0]
x_obs = np.repeat(x_obs[np.newaxis, :], mu.shape[0], axis=0)
test_data = dict(x=x_obs, mu=mu)

mu_broad = mu[:, np.newaxis, :]

ll = log_likelihood(x_obs, mu_broad)
lr = ratio_approximator.log_ratio(test_data)[:, np.newaxis]

print(ll.shape)
print(lr.shape)

print(ll - lr)

## Verify likelihood-evidence ratio with PyMC

In [ ]:
import pymc as pm
import pytensor.tensor as pt
from pytensor.graph import Op, Apply
import numpy as np

class NRECLikelihoodOp(Op):
    """
    Op for NRE-C: estimates p(x|theta) / p(x)
    """
    def __init__(self, nrec_model, x_obs):
        """
        nrec_model: your trained NRE-C classifier
        x_obs: observed data
        """
        self.model = nrec_model
        self.x_obs = x_obs[np.newaxis, :]
        
    def make_node(self, mu):
        mu = pt.as_tensor_variable(mu)
        return Apply(self, [mu], [pt.dscalar()])
    
    def perform(self, node, inputs, outputs):
        mu = inputs[0][np.newaxis, :]
        
        # NRE-C output: log(p(x|theta)/p(x))
        # This is proportional to log p(x|theta) since p(x) is constant
        data = dict(x = self.x_obs, mu = mu)
        log_ratio = self.model.log_ratio(data)
        
        outputs[0][0] = np.asarray(log_ratio[0], dtype='float64')

In [ ]:
# Usage in PyMC
nrec_op = NRECLikelihoodOp(ratio_approximator, x_obs = sims["x"][0])
nrec_op.perform(inputs = sims["mu"], node = None, outputs = [[np.empty(1)]])

In [ ]:
with pm.Model() as model:
    # prior
    mu = pm.Normal('mu', mu=0, sigma=1, shape=1)
    
    # NRE-C likelihood
    log_likelihood = nrec_op(mu)
    pm.Potential('nrec_likelihood', log_likelihood)
    
    trace = pm.sample(2000, step=pm.Metropolis(), chains = 1, cores = 1)

In [ ]:
import arviz
arviz.plot_trace(trace)